**Задание 1**

1. Используйте файл с оценками фильмов ml-latest-small/ratings.csv. Посчитайте среднее время жизни пользователей, которые выставили более 100 оценок. Под временем жизни понимается разница между максимальным и минимальным значением столбца timestamp для данного значения userId.


In [1]:
import pandas as pd

In [2]:
ratings = pd.read_csv('ml-latest-small/ratings.csv')
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [30]:
ratings_min = ratings.groupby('userId').min().reset_index()
ratings_min.rename(columns={'timestamp': 'timestamp_min'}, inplace=True)
ratings_min_2 = ratings_min[['userId', 'timestamp_min']]
ratings_min_2

,userId,timestamp_min
0,1,1260759108
1,2,835355395
2,3,1298861589
3,4,949778714
4,5,1163373044
...,...,...
666,667,847271220
667,668,993613196
668,669,1015829081
669,670,938781328


In [31]:
ratings_max = ratings.groupby('userId').max().reset_index()
ratings_max.rename(columns={'timestamp': 'timestamp_max'}, inplace=True)
ratings_max_2 = ratings_max[['userId', 'timestamp_max']]
ratings_max_2

,userId,timestamp_max
0,1,1260759205
1,2,835356246
2,3,1298932787
3,4,949982274
4,5,1163375145
...,...,...
666,667,847272234
667,668,993613478
668,669,1015829766
669,670,940944033


In [32]:
ratings_count = ratings.groupby('userId').count().reset_index()
ratings_count.rename(columns={'movieId': 'count'}, inplace=True)
ratings_count_2 = ratings_count[['userId', 'count']]
ratings_count_2

,userId,count
0,1,20
1,2,76
2,3,51
3,4,204
4,5,100
...,...,...
666,667,68
667,668,20
668,669,37
669,670,31


In [33]:
data = ratings_max_2.merge(ratings_min_2, how='left', on=['userId'])
data['lifestamp'] = data['timestamp_max'] - data['timestamp_min'] 
data.head()

,userId,timestamp_max,timestamp_min,lifestamp
0,1,1260759205,1260759108,97
1,2,835356246,835355395,851
2,3,1298932787,1298861589,71198
3,4,949982274,949778714,203560
4,5,1163375145,1163373044,2101


In [36]:
data_new = data.merge(ratings_count_2, how='left', on=['userId'])
data_new

,userId,timestamp_max,timestamp_min,lifestamp,count
0,1,1260759205,1260759108,97,20
1,2,835356246,835355395,851,76
2,3,1298932787,1298861589,71198,51
3,4,949982274,949778714,203560,204
4,5,1163375145,1163373044,2101,100
...,...,...,...,...,...
666,667,847272234,847271220,1014,68
667,668,993613478,993613196,282,20
668,669,1015829766,1015829081,685,37
669,670,940944033,938781328,2162705,31


In [58]:
data_new[data_new['count']>100].mean()

userId           3.447364e+02
timestamp_max    1.164836e+09
timestamp_min    1.124755e+09
lifestamp        4.008051e+07
count            3.119651e+02
dtype: float64

**Задание  2**

Дана статистика услуг перевозок клиентов компании по типам (см. файл с кодом занятия). Необходимо сформировать две таблицы:
- таблицу с тремя типами выручки для каждого client_id без указания адреса клиента
- аналогичную таблицу по типам выручки с указанием адреса клиента

Дана статистика услуг перевозок клиентов компании по типам:

rzd - железнодорожные перевозки
auto - автомобильные перевозки
air - воздушные перевозки
client_base - адреса клиентов

Обратите внимание, что в процессе объединения таблиц данные не должны теряться.

In [59]:
rzd = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115],
        'rzd_revenue': [1093, 2810, 10283, 5774, 981]
    }
)
rzd

,client_id,rzd_revenue
0,111,1093
1,112,2810
2,113,10283
3,114,5774
4,115,981


In [60]:
auto = pd.DataFrame(
    {
        'client_id': [113, 114, 115, 116, 117],
        'auto_revenue': [57483, 83, 912, 4834, 98]
    }
)
auto

,client_id,auto_revenue
0,113,57483
1,114,83
2,115,912
3,116,4834
4,117,98


In [61]:
air = pd.DataFrame(
    {
        'client_id': [115, 116, 117, 118],
        'air_revenue': [81, 4, 13, 173]
    }
)
air

,client_id,air_revenue
0,115,81
1,116,4
2,117,13
3,118,173


In [62]:
client_base = pd.DataFrame(
    {
        'client_id': [111, 112, 113, 114, 115, 116, 117, 118],
        'address': ['Комсомольская 4', 'Энтузиастов 8а', 'Левобережная 1а', 'Мира 14', 'ЗЖБИиДК 1', 
                    'Строителей 18', 'Панфиловская 33', 'Мастеркова 4']
    }
)
client_base

,client_id,address
0,111,Комсомольская 4
1,112,Энтузиастов 8а
2,113,Левобережная 1а
3,114,Мира 14
4,115,ЗЖБИиДК 1
5,116,Строителей 18
6,117,Панфиловская 33
7,118,Мастеркова 4


In [66]:
data = rzd.merge(auto, how='outer', on=['client_id'])
data_2 = data.merge(air, how='outer', on=['client_id'])
data_2

,client_id,rzd_revenue,auto_revenue,air_revenue
0,111,1093.0,NaN,NaN
1,112,2810.0,NaN,NaN
2,113,10283.0,57483.0,NaN
3,114,5774.0,83.0,NaN
4,115,981.0,912.0,81.0
5,116,NaN,4834.0,4.0
6,117,NaN,98.0,13.0
7,118,NaN,NaN,173.0


In [67]:
data_3 = data_2.merge(client_base, how='outer', on=['client_id'])
data_3[['address', 'rzd_revenue', 'auto_revenue', 'air_revenue']]

,address,rzd_revenue,auto_revenue,air_revenue
0,Комсомольская 4,1093.0,NaN,NaN
1,Энтузиастов 8а,2810.0,NaN,NaN
2,Левобережная 1а,10283.0,57483.0,NaN
3,Мира 14,5774.0,83.0,NaN
4,ЗЖБИиДК 1,981.0,912.0,81.0
5,Строителей 18,NaN,4834.0,4.0
6,Панфиловская 33,NaN,98.0,13.0
7,Мастеркова 4,NaN,NaN,173.0


**Задание 3**

В задаче сквозной аналитики вам предоставили данные по местоположению пользователей. Т. е. для каждого user_id известна последовательность координат (широта/долгота), когда они требовались приложению для полноценной работы. Как бы вы добавили эти сведения в таблицу визитов и покупок? Для составления ответа можно использовать вопросы:
- У каждого пользователя известен набор координат. А для связывания с визитом или фактом покупки скорее всего потребуется одно-два числа. Как их получить? - Так как пользователь может заходить и совершать покупку из одной точки несоклько раз, то для более точной аналитике необходима еще один разрез данных, например, время нахождения в точке и время визита\покупки. Тогда по времение можно будет джойнить таблицы


- Наборы координат одного и того же пользователя могут быть значительно удалены друг от друга. Как это отразится на вопросе расчетах пункта 1? - При указании времени это не должно вызвать проблем 



- Какие дополнительные признаки можно получить из координат? Ведь это просто числа, которые сами по себе мало что дают. - страну, национальность пользователей, на основе этого можно сдешлать вывод об их языке, возможно, социальном статусе (бедные/богатые страны) 
 